In [ ]:
"""
The Project
Classifying Simple Shapes (Circle vs Square)

Why this example?
Because it mirrors real tasks like:

1.Sorting components

2.Detecting defective parts

3.Classifying objects on a conveyor

And it works with images you can easily generate.
    
    
We’ll build one clean pipeline and explain each part:

1. Feature extraction

2. Train/test split

3. ML classifier on images

4. Evaluate accuracy

5. Compare classical vs ML

"""

In [ ]:
# Step 1: Feature Extraction

import cv2
import numpy as np

def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None

    cnt = max(contours, key=cv2.contourArea)

    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)

    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = w / float(h)

    circularity = 4*np.pi*area/(perimeter*perimeter) if perimeter!=0 else 0

    return [area, perimeter, aspect_ratio, circularity]


In [ ]:
# Step 2: Build Dataset

# Python Script for bing image downloader
from bing_image_downloader import downloader

# Download circles
downloader.download(
    "perfect circle white background", 
    limit=100, 
    output_dir='dataset', 
    adult_filter_off=True, 
    force_replace=False, 
    filter="clipart", # Options: line, photo, clipart, gif, transparent
    verbose=True
)

# Download squares
downloader.download(
    "perfect square white background", 
    limit=100, 
    output_dir='dataset', 
    adult_filter_off=True, 
    force_replace=False, 
    filter="clipart",
    verbose=True
)


[%] Downloading Images to f:\Python code\Opencv\dataset\perfect circle white background


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://www.freepnglogos.com/uploads/circle-png/black-and-white-round-frame-circle-png-7.png
[%] File Downloaded !

[%] Downloading Image #2 from https://storage.needpix.com/rsynced_images/white-circle.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://img.freepik.com/premium-vector/pink-white-circle-with-white-background-with-red-circle-white-background_822882-55642.jpg?w=2000
[%] File Downloaded !

[%] Downloading Image #4 from https://thumbs.dreamstime.com/b/watercolor-circle-white-background-watercolor-circle-white-as-background-169530287.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://img.freepik.com/premium-psd/orange-background-with-white-circle-white-circle_551318-1453.jpg?w=996
[%] File Downloaded !

[%] Downloading Image #6 from https://st3.depositphotos.com/1359043/32163

KeyboardInterrupt: 

In [38]:
# Python Script for compressed numpy arrays images
import numpy as np

# Saving multiple arrays
#x = np.arange(10)
#y = np.sin(x)
#np.savez('data.npz', x=x, y=y)

# Loading the file
container = np.load('shapes.npz')
print(container.files)

for key in container.files:
    print(f"Key: {key}, Shape: {container[key].shape}")

container = np.load('shapes.npz')

X_train = container['X_train']
y_train = container['y_train']
X_test  = container['X_test']
y_test  = container['y_test']

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print(X_train.shape)
print(X_train.dtype)



['X_train', 'y_train', 'X_test', 'y_test']
Key: X_train, Shape: (7500, 64, 64)
Key: y_train, Shape: (7500,)
Key: X_test, Shape: (2500, 64, 64)
Key: y_test, Shape: (2500,)
(7500, 64, 64) (7500,)
(2500, 64, 64) (2500,)
(7500, 64, 64)
float64


In [ ]:
# Icrawler script
from icrawler.builtin import GoogleImageCrawler
import os

# Define the target classes and counts
tasks = {
    'perfect circle white background': 290,
    'perfect square white background': 200
}

# Advanced Filters: Ensuring white background and clipart style
# 'color' can be: 'color', 'blackandwhite', 'red', 'orange', 'yellow', 'green', 'teal', 'blue', 'purple', 'pink', 'white', 'gray', 'black', 'brown'
filters = dict(
    type='clipart',
    color='white',
    size='medium'
)

for keyword, count in tasks.items():
    # Setup directory to match your Step 3 loading logic
    save_dir = os.path.join('dataset', keyword)
    
    # Initialize Crawler
    # downloader_threads=4 makes it much faster than bing-downloader
    crawler = GoogleImageCrawler(
        storage={'root_dir': save_dir},
        downloader_threads=4 
    )
    
    print(f"--- Starting download for: {keyword} ---")
    
    crawler.crawl(
        keyword=keyword,
        filters=filters,
        max_num=count,
        min_size=(100, 100) # Ensures we don't get tiny icons
    )

print("Dataset Download Complete!")


In [30]:
# Step 3: Load Dataset

import os

X = []
y = []

dataset_path = "dataset"

for label, folder in enumerate(["perfect circle white background","perfect square white background"]):
    folder_path = os.path.join(dataset_path, folder)

    for file in os.listdir(folder_path):
        path = os.path.join(folder_path, file)
        img = cv2.imread(path)

        features = extract_features(img)
        if features is not None:
            X.append(features)
            y.append(label)



In [31]:
# Step 3: Train/Test Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [32]:
# Step 4: ML Classifier

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [33]:
# Step 5: Evaluate Accuracy

from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5714285714285714


In [ ]:
img = cv2.imread("dataset/img/square.jpg")
features = extract_features(img)

prediction = model.predict([features])

if prediction[0] == 0:
    print("Circle")
else:
    print("Square")


Square


In [36]:
import cv2
import numpy as np

# 1. Initialize Webcam
cap = cv2.VideoCapture(0)

print("Starting Robot Vision... Press 'q' to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- DETECTION LOGIC (Finding the object) ---
    # We use the same preprocessing as your 'extract_features' function
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Blur helps the webcam handle flickering lights
    blurred = cv2.GaussianBlur(gray, (25, 25), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        # Ignore tiny noise/specks
        if cv2.contourArea(cnt) < 1000:
            continue

        # --- FEATURE EXTRACTION (Objective 1) ---
        # We must extract the EXACT same features used to train the model
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter != 0 else 0

        # Create the feature vector for the model
        current_features = [area, perimeter, aspect_ratio, circularity]

        # --- CLASSIFICATION (Objective 2) ---
        # The model expects a 2D array: [[f1, f2, f3, f4]]
        prediction = model.predict([current_features])
        
        # Mapping numerical labels back to text
        if prediction[0] == 0:
            label = "Circle"
            color = (0, 255, 0) # Green for circle
        else:
            label = "Square"
            color = (255, 0, 0) # Blue for square

        # --- VISUALIZATION ---
        # Draw the bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        # Draw the label text
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Display the result
    cv2.imshow("Robot Perception - Live Classification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Starting Robot Vision... Press 'q' to exit.
